## 1.背包问题
* W数组：物件重量，V数组：物件价值，bag整数：背包总大小.
* 返回不超过背包大小下的最大价值

In [4]:
import numpy as np

In [5]:
def maxValue(w, v, bag):
    if not w or not v or len(w)!=len(v):
        return 0
    return process(w,v,0,bag)
#考虑到了index号货物，index...最后一个货物可以自由选择
#做的选择不超过背包容量，返回最大价值
def process(w, v, index, res):
    if res < 0: #w可能有0
        return -1
    if index == len(w): #没货了
        return 0
    #有货，index位置的货；背包有地，>=0
    p1 = process(w, v, index+1, res) #没要index
    p2 = 0
    next_value = process(w, v, index+1, res-w[index]) #要了index
    if next_value != -1: #basecase调整，后续返回-1，告知无效
        p2 = v[index] + next_value
    
    return max(p1,p2)
    

In [6]:
def dp(w,v, bag):
    if not w or not v or len(w)!=len(v):
        return 0
    N = len(w)
    dp = np.ones((N+1, bag+1)) * (-1)
    #对应if index == len(w): return 0 #没货了
    for i in range(bag+1):
        dp[N][i] = 0
    for index in range(N-1, -1, -1):
        for rest in range(bag+1):
#       对应p1 = process(w, v, index+1, res) #没要index
            p1 = dp[index+1][rest]
         #对应p2 = 0
#          next_value = process(w, v, index+1, res-w[index]) #要了index
#          if next_value != -1: #basecase调整，后续返回-1，告知无效
#          p2 = v[index] + next_value
            p2 = 0
            if rest-w[index] >=0:
                p2 = dp[index+1][rest-w[index]] + v[index]
            dp[index][rest] = max(p1,p2)
    return dp[0][bag]

In [7]:
w = [3,2,4, 7, 3,1,7]
v = [5,6,3,19,12,4,2]
bag = 15
print(maxValue(w,v,bag))
print(dp(w,v,bag))

42
42.0


## 2. 数字转换字符串
* 1对应A，2对应B，3对应C...26对应Z
* 那么一个数字字符串比如“111”可以转化为“AAA”、“KA”和”AK“
* 给定一个数组的字符串，返回有多少转化结果

In [35]:
# string[0...i-1]已转化
# string[i,....]转化问题
def process(listString, i):
    if i == len(listString):
        return 1
    #i没到最后，说明有字符
    #如果i位置上是0，则前面转化决策有问题：0不对应字符，只能在前面是10转化
    if listString[i] == "0":
        return 0
    #i不是0，总有决定可以单独转化
    ways = process(listString, i+1)
    #i不是0，i+1不越界且i、i+1两个组成的不能>26
    if i + 1 < len(listString) and ((ord(listString[i]) - ord("0")) * 10 + (ord(listString[i+1]) - ord("0"))) <= 26:
        ways += process(listString, i+2)
    return ways
def numberTrans(string):
    if not string:
        return 0
    return process(list(string), 0)

In [41]:
print(numberTrans("7230231231232031203123"))
print(numberTrans("11111"))
print(numberTrans("111"))

0
8
3


In [44]:
def dp(string):
    if not string:
        return 0
    listString = list(string)
    N = len(listString)
    dp = [-1] * (N+1)
    dp[N] = 1
    for i in range(N-1, -1, -1):
        if listString[i] == "0":
            dp[i] = 0
        else:
            ways = dp[i+1]
            if i + 1 < len(listString) and ((ord(listString[i]) - ord("0")) * 10 + (ord(listString[i+1]) - ord("0"))) <= 26:
                ways += dp[i+2]
            dp[i] = ways
    return dp[0]

In [46]:
print(dp("7230231231232031203123"))
print(dp("11111"))
print(dp("111"))

0
8
3


## 3. 贴纸拼字符
* 给定一个字符串str，一个字符串类型的数组arr，出现的字符都是小写英文。
* arr的每一个字符串代表一张贴纸，这张贴纸可以把每个字符剪开使用，目的是拼出str。给的某一种贴纸有无穷张。
* 返回至少多少张贴纸可以完成这个任务。
* 例如 str=babac， arr=[ba, c, abcd]
* 返回 2. 因为需要ba、abcd，单独剪开后获得2个a、2个b、1个c，可以拼出str
* 链接：https://leetcode.com/problems/stickers-to-spell-word/

In [59]:
#所有贴纸stickers，每一种贴纸有无穷张
#目标拼出target
#返回最少张数
def process1(stickers, target):
    #目标不剩了
    if len(target)==0:
        return 0
    min_old = None
    for first in stickers:
        rest = minus(target, first)
        #可以有效减去>0个字符
        if len(rest) != len(target):
            min_new = process1(stickers, rest)
            if min_old is None:
                min_old = min_new
            if min_new is not None:
                min_old = min(min_old, min_new)
    if min_old is None:
        return None
    return min_old + 1

def minus(target, sticker):
    t = list(target)
    s = list(sticker)
    count = [0] * 26
    for cha in t:
        count[ord(cha) - ord("a")] += 1
    for cha in s:
        count[ord(cha) - ord("a")] -= 1
    res = ""
    for i in range(26):
        if count[i] > 0:
            for j in range(count[i]):
                res += chr(i+ord("a"))
    return res

def minStickers(stickers, target):
    ans = process1(stickers, target)
    if ans is None:
        ans = -1
    return ans

In [79]:
#仍然超时！
#所有贴纸stickers[][]，每一种贴纸有无穷张: stickers[i]代表i号贴纸的字符频统计
#目标拼出target
#返回最少张数
def process2(stickers, target):
    #目标不剩了
    if len(target)==0:
        return 0
    #target字符频统计
    tcounts = [0] * 26
    for cha in target:
        tcounts[ord(cha)-ord("a")]+=1
    
    N = len(stickers)
    min_old = None
    for i in range(N):
        #逐个让target的字符频减去当前贴纸字符频
        sticker = stickers[i]
        #关键优化，只有贴纸里存在target第一个字符的试， 剪枝
        if sticker[ord(target[0]) - ord("a")] > 0:
            rest = ""
            for j in range(26):
                if tcounts[j] > 0:
                    nums = tcounts[j] - sticker[j]
                    for k in range(int(nums)):
                        rest += chr(j + ord("a"))
            min_new = process2(stickers, rest)
            if min_old is None:
                min_old = min_new
            if min_new is not None:
                min_old = min(min_old, min_new) 
    if min_old is None:
        return None
    return min_old + 1



def minStickers2(stickers, target):
    N = len(stickers)
    counts = np.zeros((N, 26))
    for i in range(N):
        stick = stickers[i]
        for c in stick:
            counts[i][ord(c)-ord("a")]+=1
    ans = process2(counts, target)
    if ans is None:
        ans = -1
    return ans

In [102]:
#如果超时，可能是哪里没有加缓存！
#只需要改傻缓存法，因为可变参数是字符串，很难用下标范围搞定
def process3(stickers, target, dp):
    if target in dp:
        return dp[target]
    #没算过
    #目标不剩了
    if len(target)==0:
        dp[target] = 0
        return 0
    #target字符频统计
    tcounts = [0] * 26
    for cha in target:
        tcounts[ord(cha)-ord("a")]+=1
    N = len(stickers)
    min_old = -1
    for i in range(N):
        #逐个让target的字符频减去当前贴纸字符频
        sticker = stickers[i]
        #关键优化，只有贴纸里存在target第一个字符的试， 剪枝
        if sticker[ord(target[0]) - ord("a")] > 0:
            rest = ""
            for j in range(26):
                if tcounts[j] > 0:
                    nums = tcounts[j] - sticker[j]
                    if nums > 0:
                        rest += chr(int(j + ord("a"))) * nums
            min_new = process3(stickers, rest, dp)
            if min_old == -1:
                min_old = min_new
            if min_new != -1:
                min_old = min(min_old, min_new) 
    if min_old == -1:
        dp[target] = -1
        return -1
    dp[target] = min_old + 1 #别忘了赋值！！！
    return min_old + 1
    
    
def minStickers3(stickers, target):
    N = len(stickers)
    counts = np.zeros((N, 26))
    for i in range(N):
        stick = stickers[i]
        for c in stick:
            counts[i][ord(c)-ord("a")]+=1
    dp = dict()
    ans = process3(counts, target, dp)
    return ans

## 4.最长公共子序列
* 子序列可以不连续
* 链接：https://leetcode.com/problems/longest-common-subsequence/

In [111]:
#str1[0...i]
#str2[0...j] 最长多少？
def process1(str1, str2, i, j):
    if i == 0 and j == 0:
        return 1 if str1[i] == str2[j] else 0
    elif i == 0:
        if str1[i] == str2[j]:
            return 1
        else:
            return process1(str1, str2, i, j-1)
    elif j == 0:
        if str1[i] == str2[j]:
            return 1
        else:
            return process1(str1, str2, i-1, j)
    else: #i,j都不是0
        #如果考虑必须以i结尾、不以j结尾，就需要遍历j中有没有[i]，增加了要求。（可以自己尝试
        #所以”考虑“可能是结尾，让递归方便进入下一级
        #情况1， 公共子序列不以i结尾，可能是j结尾
        p1 = process1(str1, str2, i-1, j)
        #情况2， 公共子序列可能是i结尾，不以j结尾
        p2 = process1(str1, str2, i, j-1)
        #情况1、2有重叠，即不以i、j结尾，无所谓，考虑的是最长的
        #情况3， 必须以i、j结尾，要求[i]=[j]
        if str1[i] == str2[j]:
            p3 = process1(str1, str2, i-1, j-1) + 1
        else:
            p3 = 0
        return max([p1,p2,p3])
        

def longestCommonSubsequence1(s1, s2):
    if not s1 or not s2:
        return 0
    str1 = list(s1)
    str2 = list(s2)
    return process1(str1, str2, len(str1)-1, len(str2)-1)

In [115]:
#str1[0...i]
#str2[0...j] 最长多少？
def process1_1(str1, str2, i, j):
    if i == 0 and j == 0:
        return 1 if str1[i] == str2[j] else 0
    elif i == 0:
        if str1[i] == str2[j]:
            return 1
        else:
            return process1_1(str1, str2, i, j-1)
    elif j == 0:
        if str1[i] == str2[j]:
            return 1
        else:
            return process1_1(str1, str2, i-1, j)
    else: #i,j都不是0
        #如果考虑必须以i结尾、不以j结尾，就需要遍历j中有没有[i]，增加了要求。（可以自己尝试
        #（自己尝试发现，实际可以不判断是否存在中，因为每次有循环j-1或i-1，如果减到0且不等，会返回0
        #情况1， 公共子序列以i结尾，不以j结尾
        if str1[i] in str2[:j]:
            p1 = process1_1(str1, str2, i, j-1)
        else:
            p1 = 0
        #情况2， 公共子序列不以i结尾，以j结尾
        if str2[j] in str1[:i]:
            p2 = process1_1(str1, str2, i-1, j)
        else:
            p2 = 0
        #情况3， 必须以i、j结尾，要求[i]=[j]
        if str1[i] == str2[j]:
            p3 = process1_1(str1, str2, i-1, j-1) + 1
        else:
            p3 = 0
        #情况4，不以i、j结尾：
        p4 = process1_1(str1, str2, i-1, j-1)
        return max([p1,p2,p3,p4])
        

def longestCommonSubsequence1_1(s1, s2):
    if not s1 or not s2:
        return 0
    str1 = list(s1)
    str2 = list(s2)
    return process1_1(str1, str2, len(str1)-1, len(str2)-1)

In [116]:
a = "abcde"
b = "ace"
print(longestCommonSubsequence1(a,b))
print(longestCommonSubsequence1_1(a,b))

3
3


In [123]:
def longestCommonSubsequence2(s1, s2):
    if not s1 or not s2:
        return 0
    str1 = list(s1)
    str2 = list(s2)
    N = len(str1)
    M = len(str2)
    dp = np.zeros((N,M))
    dp[0][0] = 1 if str1[0] == str2[0] else 0
    for j in range(1, M):
        dp[0][j] = 1 if str1[0] == str2[j] else dp[0][j-1]
    for i in range(1, N):
        dp[i][0] = 1 if str1[i] == str2[0] else dp[i-1][0]
    for i in range(1, N):
        for j in range(1, M):
            p1 = dp[i-1][j]
        #情况2， 公共子序列可能是i结尾，不以j结尾
            p2 = dp[i][j-1]
        #情况1、2有重叠，即不以i、j结尾，无所谓，考虑的是最长的
        #情况3， 必须以i、j结尾，要求[i]=[j]
            if str1[i] == str2[j]:
                p3 = dp[i-1][j-1] + 1
            else:
                p3 = 0
            dp[i][j] = max([p1,p2,p3])
    return int(dp[N-1][M-1])

In [125]:

def longestCommonSubsequence2_1(s1, s2):
    if not s1 or not s2:
        return 0
    str1 = list(s1)
    str2 = list(s2)
    N = len(str1)
    M = len(str2)
    dp = np.zeros((N,M))
    dp[0][0] = 1 if str1[0] == str2[0] else 0
    for j in range(1, M):
        dp[0][j] = 1 if str1[0] == str2[j] else dp[0][j-1]
    for i in range(1, N):
        dp[i][0] = 1 if str1[i] == str2[0] else dp[i-1][0]
    for i in range(1, N):
        for j in range(1, M):
            p1 = dp[i][j-1]
            p2 = dp[i-1][j]
            if str1[i] == str2[j]:
                p3 = dp[i-1][j-1] + 1
            else:
                p3 = 0
            p4 = dp[i-1][j-1]
            dp[i][j] = max([p1,p2,p3,p4])
        
    return dp[N-1][M-1]

In [126]:
longestCommonSubsequence2(a,b)

3

In [105]:
max([1,2,3])

3

In [96]:
stickers = ["with","example","science"]
target = "thehat"
minStickers3(stickers, target)

3

In [100]:
for i in "a":
    print(i)

a


In [101]:
res = ""
res+="abbb"
a = dict()

In [83]:
a["1"] = 1

In [85]:
a[""] = 1

In [87]:
len("")

0

In [48]:
l = ["a","b"]

In [49]:
l.remove(0)

ValueError: list.remove(x): x not in list

In [51]:
ord("a")

97

In [54]:
chr(97)

'a'